# <a href="http://www.datascience-paris-saclay.fr">Paris Saclay Center for Data Science</a>
# <a href=https://ramp.r0h.eu/problems/air_passengers>RAMP</a> on predicting the number of air passengers

<i> Balázs Kégl (LAL/CNRS), Alex Gramfort (LTCI/Telecom ParisTech), Djalel Benbouzid (UPMC), Mehdi Cherti (LAL/CNRS) </i>

## Introduction
The data set was donated to us by an unnamed company handling flight ticket reservations. The data is thin, it contains
<ul>
<li> the date of departure
<li> the departure airport
<li> the arrival airport
<li> the mean and standard deviation of the number of weeks of the reservations made before the departure date
<li> a field called <code>log_PAX</code> which is related to the number of passengers (the actual number were changed for privacy reasons)
</ul>

The goal is to predict the <code>log_PAX</code> column. The prediction quality is measured by RMSE. 

The data is obviously limited, but since data and location informations are available, it can be joined to external data sets. <b>The challenge in this RAMP is to find good data that can be correlated to flight traffic</b>.

To do :
- mean encoding 
- regrouper variables catégoriques
- ajouter nouvelles données : distance aéroports (trouver fichier avec données)
- faire analyses descriptives

In [1]:
%matplotlib inline
import imp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

## Fetch the data and load it in pandas

First we load `problem.py` that parameterizes the challenge. It contains some objects taken off the shelf from `ramp-workflow` (e.g., `Predictions` type, scores, and data reader). 

In [2]:
problem = imp.load_source('', 'problem.py')

`get_train_data` loads the training data and returns an `pandas` object (input) and a `np.array` object (output).

In [3]:
X_df, y_array = problem.get_train_data()
# the y array contains the log-like number of passengers on each flight 

In [4]:
print(min(X_df['DateOfDeparture']))
print(max(X_df['DateOfDeparture']))

2011-09-01
2013-03-05


In [7]:
X_df['log_PAX'] = y_array
cols = X_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
X_df = X_df[cols]
X_encoded = X_df
X_encoded.head()

,WeeksToDeparture,std_wtd,log_PAX,DateOfDeparture,Departure,Arrival
0,12.875000,9.812647,12.331296,2012-06-19,ORD,DFW
1,14.285714,9.466734,10.775182,2012-09-10,LAS,DEN
2,10.863636,9.035883,11.083177,2012-10-05,DEN,LAX
3,11.480000,7.990202,11.169268,2011-10-09,ATL,ORD
4,11.450000,9.517159,11.269364,2012-02-21,DEN,SFO


In [ ]:
data = pd.read_csv('submissions/starting_kit/external_data.csv', sep=';')

In [ ]:
X_encoded = X_df
external_data = data[['DateOfDeparture', 'Departure','Arrival', 'Mean TemperatureC', 'Distance',
                      'MeanDew PointC', 'Mean Humidity', 'Min VisibilitykM', 'Mean Sea Level PressurehPa',
                      'Max Wind SpeedKm/h', 'Precipitationmm', 'CloudCover',
                      'Events','dep_encod','ar_encod','Number_hab','Revenue']]
X_encoded = pd.merge(
            X_encoded, external_data, how='left',
            left_on=['DateOfDeparture', 'Arrival','Departure'],
            right_on=['DateOfDeparture', 'Arrival','Departure'],
            sort=False)
X_encoded.head()

In [ ]:
X_encoded.drop(['std_wtd'], axis=1, inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler

X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Departure'], prefix='d'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Arrival'], prefix='a'))
X_encoded = X_encoded.drop('Departure', axis=1)
X_encoded = X_encoded.drop('Arrival', axis=1)

X_encoded['DateOfDeparture'] = pd.to_datetime(X_encoded['DateOfDeparture'])

X_encoded['year']    = X_encoded['DateOfDeparture'].dt.year
X_encoded['month']   = X_encoded['DateOfDeparture'].dt.month
X_encoded['day']     = X_encoded['DateOfDeparture'].dt.day
X_encoded['weekday'] = X_encoded['DateOfDeparture'].dt.weekday
X_encoded['week']    = X_encoded['DateOfDeparture'].dt.week
X_encoded['n_days']  = X_encoded['DateOfDeparture'].apply(lambda date: (date - pd.to_datetime("1970-01-01")).days)

X_encoded = X_encoded.join(pd.get_dummies(X_encoded['year'], prefix='y'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['month'], prefix='m'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['day'], prefix='d'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['weekday'], prefix='wd'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['week'], prefix='w'))




In [ ]:
X_encoded[['day', 'log_PAX']].groupby(['day']).mean().sort_values(by='log_PAX', ascending=False)

In [ ]:
X_encoded[['year', 'log_PAX']].groupby(['year']).mean().sort_values(by='log_PAX', ascending=False)

In [ ]:
X_encoded[['month', 'log_PAX']].groupby(['month']).mean().sort_values(by='log_PAX', ascending=False)

In [ ]:
X_encoded[['week', 'log_PAX']].groupby(['week']).mean().sort_values(by='log_PAX', ascending=False)

In [ ]:
X_encoded[['weekday', 'log_PAX']].groupby(['weekday']).mean().sort_values(by='log_PAX', ascending=False)

In [ ]:
X_encoded.drop(['year', 'month', 'day', 'weekday', 'week', 'n_days','DateOfDeparture'], axis=1, inplace=True)

In [ ]:
X_encoded['Events'].fillna(0, inplace=True)
X_encoded['Events'] = X_encoded['Events'].replace(['Rain','Fog'], 1)
X_encoded['Events'] = X_encoded['Events'].replace(['Rain-Thunderstorm','Fog-Rain-Thunderstorm', 
                                                         'Rain-Snow','Snow','Fog-Rain','Thunderstorm','Fog-Snow', 
                                                         'Fog-Rain-Snow','Fog-Rain-Snow-Thunderstorm', 
                                                         'Rain-Snow-Thunderstorm','Rain-Hail-Thunderstorm', 
                                                         'Fog-Rain-Hail-Thunderstorm', 
                                                         'Rain-Thunderstorm-Tornado'], 2)

In [ ]:
X_encoded['Precipitationmm'] = X_encoded['Precipitationmm'].replace('T', np.nan)
X_encoded['Precipitationmm'] = pd.to_numeric(X_encoded['Precipitationmm'])
X_encoded['Precipitationmm'] = X_encoded['Precipitationmm'].fillna(X_encoded['Precipitationmm'].mean())

In [ ]:
X_encoded['Precipitationmm*CloudCover'] = X_encoded.Precipitationmm * X_encoded.CloudCover
X_encoded['Temperature*Humidity'] = X_encoded['Mean TemperatureC'] * X_encoded['Mean Humidity']

In [8]:
X_encoded['Weeks_to_dep_int'] = pd.qcut(X_encoded['WeeksToDeparture'], 4)
X_encoded.loc[X_encoded['WeeksToDeparture'] <= 9.524, 'WeeksToDeparture'] = 0
X_encoded.loc[(X_encoded['WeeksToDeparture'] > 9.524) & (X_encoded['WeeksToDeparture'] <= 11.3), 'WeeksToDeparture'] = 1
X_encoded.loc[(X_encoded['WeeksToDeparture'] > 11.3) & (X_encoded['WeeksToDeparture'] <= 13.24), 'WeeksToDeparture'] = 2
X_encoded.loc[ X_encoded['WeeksToDeparture'] > 13.24, 'WeeksToDeparture'] = 3
X_encoded['WeeksToDeparture'] = X_encoded['WeeksToDeparture'].astype(int)
X_encoded.drop(['Weeks_to_dep_int'], axis=1, inplace=True)

In [10]:
X_encoded[['WeeksToDeparture', 'log_PAX']].groupby(['WeeksToDeparture']).mean().sort_values(by='log_PAX', ascending=False)

,log_PAX
WeeksToDeparture,
3,11.213970
2,11.010436
1,10.934104
0,10.838703


In [ ]:
X_encoded.loc[:, 'Precipitationmm*CloudCover' : 'Temperature*Humidity'] = StandardScaler().fit_transform(
                                                            X_encoded.loc[:, 'Precipitationmm*CloudCover' : 'Temperature*Humidity'])

X_encoded.loc[:, 'Distance' : 'Max Wind SpeedKm/h'] = StandardScaler().fit_transform(
                                                            X_encoded.loc[:, 'Distance' : 'Max Wind SpeedKm/h'])

X_encoded.loc[:, 'dep_encod' : 'Revenue'] = StandardScaler().fit_transform(
                                                            X_encoded.loc[:, 'dep_encod' : 'Revenue'])

In [ ]:
X_encoded.drop(['Mean TemperatureC','Mean Humidity','Precipitationmm','CloudCover'], axis=1, inplace=True)
X_encoded.drop(['MeanDew PointC', 'Mean Sea Level PressurehPa'], axis=1, inplace=True)
X_encoded.drop(['ar_encod'], axis=1, inplace=True)

In [ ]:
y = X_encoded['log_PAX']
X_encoded.drop(['log_PAX'], axis=1, inplace=True)

In [ ]:
X_encoded.head()

In [ ]:
import xgboost as xgb
data_dmatrix = xgb.DMatrix(data=X_encoded,label=y)

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

X_train, X_test, Y_train, Y_test = train_test_split(X_encoded, y, test_size=0.2)

In [ ]:
#Gridsearch
import xgboost as xgb
xgb1 = xgb.XGBRegressor()
parameters = {'nthread':[4],
              'objective':['reg:squarederror'],
              'learning_rate': [.2], 
              'max_depth': [7],
              'min_child_weight': [4],
              'subsample': [0.95],
              'colsample_bytree': [0.55],
              'n_estimators': [100,300,500,1000],
              'reg_lambda':[0.001],
              'gamma':[0]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(X_train, Y_train)

print(xgb_grid.best_params_)

In [ ]:
parameters = {'nthread':4,
              'objective':'reg:squarederror',
              'learning_rate': .3, 
              'max_depth': 7,
              'min_child_weight': 4,
              'subsample': 0.95,
              'colsample_bytree': 0.55,
              'n_estimators': 1000,
              'reg_lambda':0.001,
              'gamma':0}

cv_results = xgb.cv(dtrain=data_dmatrix, params=parameters, nfold=3,
                    num_boost_round=300,early_stopping_rounds=5,metrics="rmse", as_pandas=True, seed=123)

In [ ]:
cv_results.head()

In [ ]:
print((cv_results["train-rmse-mean"]).tail(1))
print((cv_results["test-rmse-mean"]).tail(1))

In [ ]:
xg_reg = xgb.train(params=parameters, dtrain=data_dmatrix, num_boost_round=10)
xgb.plot_importance(xg_reg, max_num_features=40)
plt.rcParams['figure.figsize'] = [20, 20]
plt.show()

In [ ]:
import lightgbm as lgb 

In [ ]:
from lightgbm import LGBMRegressor
lightgbm = LGBMRegressor(task= 'train',
          boosting_type= 'gbdt',
          objective= 'regression',
          metric= {'l2','auc'},
          num_leaves= 300,
          learning_rate= 0.32,
          feature_fraction= 0.9,
          bagging_fraction= .9,
          bagging_freq= 70,
          verbose= 100)

In [ ]:
cv_results = lgb.cv(dtrain=data_dmatrix, params=parameters, nfold=3,
                    num_boost_round=300,early_stopping_rounds=5,metrics="rmse", as_pandas=True, seed=123)